![title](CBMpy.png)

**INSTITUTO NACIONAL DE PESQUISAS ESPACIAIS** 

Disciplina: Introdution to Data Science
    
Professores: Rafael Santos e Gilberto Queiroz
    
Acadêmica: Marcelly Homem Coelho
    
Contato: marcellyhc@gmail.com 

**Título:** Aplicação de Técnicas de Data Science no Desenvolvimento de um Sistema para Manutenção Aeronáutica Baseada em Condição 

**Descrição:** Este programa tem como objetivo analisar as mensagens de falha e as remoções dos sistemas das aeronaves.

In [21]:
# Importar as bibliotecas

import numpy as np
import pandas as pd
import seaborn as sns

import random

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
# Função generate_color é utilizada para gerar cores aleatórias

def generate_color():
    color = '#{:02x}{:02x}{:02x}'.format(*map(lambda x: random.randint(0, 255), range(3)))
    return color

# 1. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Falha

In [23]:
# Criar um dataframe para entrada de dados de falha (arquivo do tipo .csv)

df_Failure = pd.read_csv('CBMpy_dataFailureCode.csv')   

In [24]:
# Exibir o cabeçalho do dataframe
    
df_Failure.head()

,Unnamed: 0,Aircraft,Flight Phase,Date,Fault Text,Report Maintenance
0,0,2640,NaN,2006-05-14 16:19:00,FDE_Other02,NaN
1,1,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_B_System3,RMA_A_System3
2,2,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_C_System3,RMA_A_System3
3,3,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_B_System3,RMA_A_System3
4,4,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_C_System3,RMA_A_System3


$\color{blue}{\text{OBSERVAÇÃO:}}$ 

A Fig. 1 destaca o Sistema de Indicação do Motor e Alerta de Tripulação - Engine Indication and Crew Alerting System (EICAS) - é definido como um sistema da aeronave responsável por exibir parâmetros do motor e alertar a tripulação sobre a configuração ou falhas do sistema.
No EICAS existe três modos de exibição de informações, são eles: 
    - Modo operacional: apresenta as informações operacionais e alertas que exigem ação da tripulação em voo.
    - Modo de status: exibe parâmetros de subsistemas e mensagens de status dos equipamentos/componentes. No df_dataFailure é representado pela coluna 'Fault Text'. 
    - Modo de manutenção: fornece aos responsáveis pela manutenção informações para auxiliar na detecção de falhas e nos testes de verificação de subsistemas. No df_dataFailure é representado pela coluna 'Maintenance Message'.  

![title](FDE_CAS.png)

Fig. 1: Crew Alerting System.

$\color{blue}{\text{OBSERVAÇÃO:}}$ 

As aeronaves possuem um Painel de Controle de Manutenção, conforme a Fig. 2, é usado por responsáveis pela manutenção para exibir dados de manutenção armazenados na memória. No df_dataFailure é representado pela coluna 'Maintenance Message'.

![title](PainelControleManutenção.png)

Fig. 2: Maintenance Control Panel .

In [25]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_Failure.shape

(7238, 6)

In [26]:
# Verificar o tipo de dado de cada coluna do dataframe

df_Failure.dtypes

Unnamed: 0             int64
Aircraft               int64
Flight Phase          object
Date                  object
Fault Text            object
Report Maintenance    object
dtype: object

In [27]:
# Converter os dados da coluna 'Date' para o formato de data-hora

df_Failure['Date'] =  pd.to_datetime(df_Failure['Date'], format='%Y/%m/%d %H:%M')

In [28]:
# Verificar o tipo de dado de cada coluna do dataframe

df_Failure.dtypes

Unnamed: 0                     int64
Aircraft                       int64
Flight Phase                  object
Date                  datetime64[ns]
Fault Text                    object
Report Maintenance            object
dtype: object

In [29]:
# Determinar quantas 'Flight Phase' diferentes há no dataframe

len(df_Failure['Aircraft'].unique())

15

In [30]:
df_Failure['Aircraft'].value_counts()

2766    1059
1950     728
1990     668
1151     626
2640     560
326      475
131      465
2436     421
1419     419
791      417
2838     369
2209     324
312      268
1710     243
2982     196
Name: Aircraft, dtype: int64

$\color{blue}{\text{OBSERVAÇÃO:}}$ O Aircraft igual a 2766 é o que apresenta a maior quantidade de Fault Text. Além disso, é o segundo em relação a quantidade de remoções de peças.

## 1.1 Agrupamento do Conjunto de Dados de Fault Text para todos os Aircraft

Este dataframe será usado posteriormente para detectar as concentrações de todas as Fault Text de todos os Aircraft.

In [31]:
# Exibir o cabeçalho do dataframe

df_Failure.head()

,Unnamed: 0,Aircraft,Flight Phase,Date,Fault Text,Report Maintenance
0,0,2640,NaN,2006-05-14 16:19:00,FDE_Other02,NaN
1,1,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_B_System3,RMA_A_System3
2,2,2640,Enroute Cruise,2006-07-01 15:17:00,FDE_C_System3,RMA_A_System3
3,3,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_B_System3,RMA_A_System3
4,4,2640,Enroute Cruise,2006-07-02 04:48:00,FDE_C_System3,RMA_A_System3


In [32]:
# Identifica todas as "Fault Text" existentes de todos os Aircraft
array_FDE_All = np.array(df_Failure['Fault Text'].unique())

# Exclui os itens NaN do array
array_FDE_All = array_FDE_All[~pd.isnull(array_FDE_All)]  

# Exibe o array
array_FDE_All

array(['FDE_Other02', 'FDE_B_System3', 'FDE_C_System3', 'FDE_C_System4',
       'FDE_B_System4', 'FDE_G_System3', 'FDE_E_System4', 'FDE_A_System4',
       'FDE_F_System4', 'FDE_G_System4', 'FDE_F_System3', 'FDE_E_System1',
       'FDE_A_System1', 'FDE_A_System3', 'FDE_D_System3', 'FDE_Other10',
       'FDE_Other11', 'FDE_Other12', 'FDE_G_System2', 'FDE_Other03',
       'FDE_Other00', 'FDE_Other04', 'FDE_E_System3', 'FDE_Other01',
       'FDE_G_System1', 'FDE_F_System1', 'FDE_Other09', 'FDE_Other05',
       'FDE_D_System4', 'FDE_H_System4', 'FDE_H_System1', 'FDE_C_System2',
       'FDE_B_System2', 'FDE_D_System1', 'FDE_D_System2', 'FDE_Other06',
       'FDE_K_System1', 'FDE_K_System4', 'FDE_K_System2', 'FDE_Q_System3',
       'FDE_C_System1', 'FDE_A_System2', 'FDE_E_System2', 'FDE_H_System3',
       'FDE_M_System3', 'FDE_F_System2', 'FDE_Other18', 'FDE_B_System1',
       'FDE_Other17', 'FDE_J_System3', 'FDE_Other16', 'FDE_Other20',
       'FDE_Other21', 'FDE_I_System2', 'FDE_I_System3',

In [33]:
arrayAux = []

df_FDE_All = pd.DataFrame(columns= ['Aircraft','year', 'month', 'day'])

for aux in array_FDE_All:
    
    # Cria um dataframe para cada Fault Text corrente 
    dfMsg = pd.DataFrame(df_Failure[df_Failure['Fault Text'] == aux])   
    
    # Contagem de Fault Text por dia para um determinado tail
    arrayAux = dfMsg.groupby([dfMsg['Aircraft'],
                              dfMsg['Date'].dt.year.rename('year'),
                              dfMsg['Date'].dt.month.rename('month'),
                              dfMsg['Date'].dt.day.rename('day')]).count()['Fault Text']
    
    # Transforma os objetos do groupby para dataframe, depois possibilita fazer o merge
    arrayAux = arrayAux.to_frame().reset_index()
    
    arrayAux.columns = ['Aircraft', 'year', 'month', 'day', aux]
    
    # Realiza o merge dos dataframes, utilizando o método "outer" apropriado para acrescentar colunas
    # e manter os índices compostos de dia-mes-ano conforme referência na documentação
    # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html  
    df_FDE_All = pd.merge(df_FDE_All, arrayAux, how='outer', on=['Aircraft', 'year','month','day'])

In [34]:
# Exibir o cabeçalho do dataframe

df_FDE_All.head()

,Aircraft,year,month,day,FDE_Other02,FDE_B_System3,FDE_C_System3,FDE_C_System4,FDE_B_System4,FDE_G_System3,...,FDE_M_System2,FDE_Other19,FDE_J_System4,FDE_Other14,FDE_P_System4,FDE_O_System2,FDE_O_System3,FDE_J_System2,FDE_L_System1,FDE_J_System1
0,131,2006,8,15,6.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131,2006,8,20,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131,2006,8,24,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,131,2006,9,7,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,131,2007,10,6,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Adicionar uma coluna date no dataframe (coo dados dos campos year, month e day) 

df_FDE_All['Date'] = pd.to_datetime(df_FDE_All.year*10000 + df_FDE_All.month*100 + df_FDE_All.day, format='%Y%m%d') 

In [42]:
# Ordena o dataframe por: year -> month -> day

df_FDE_All = df_FDE_All.sort_values(['year', 'month', 'day'])

In [37]:
# Substituir elementos NaN por zeros 

df_FDE_All = df_FDE_All.fillna(0) 

In [38]:
df_FDE_All.columns

Index(['Aircraft', 'year', 'month', 'day', 'FDE_Other02', 'FDE_B_System3',
       'FDE_C_System3', 'FDE_C_System4', 'FDE_B_System4', 'FDE_G_System3',
       'FDE_E_System4', 'FDE_A_System4', 'FDE_F_System4', 'FDE_G_System4',
       'FDE_F_System3', 'FDE_E_System1', 'FDE_A_System1', 'FDE_A_System3',
       'FDE_D_System3', 'FDE_Other10', 'FDE_Other11', 'FDE_Other12',
       'FDE_G_System2', 'FDE_Other03', 'FDE_Other00', 'FDE_Other04',
       'FDE_E_System3', 'FDE_Other01', 'FDE_G_System1', 'FDE_F_System1',
       'FDE_Other09', 'FDE_Other05', 'FDE_D_System4', 'FDE_H_System4',
       'FDE_H_System1', 'FDE_C_System2', 'FDE_B_System2', 'FDE_D_System1',
       'FDE_D_System2', 'FDE_Other06', 'FDE_K_System1', 'FDE_K_System4',
       'FDE_K_System2', 'FDE_Q_System3', 'FDE_C_System1', 'FDE_A_System2',
       'FDE_E_System2', 'FDE_H_System3', 'FDE_M_System3', 'FDE_F_System2',
       'FDE_Other18', 'FDE_B_System1', 'FDE_Other17', 'FDE_J_System3',
       'FDE_Other16', 'FDE_Other20', 'FDE_Other21

In [43]:
# Preenche as datas faltantes 

df_FDE_All['Date'] = pd.to_datetime(df_FDE_All.Date)

df_FDE_All = df_FDE_All.set_index('Date')

df_FDE_All = df_FDE_All.groupby('Aircraft').resample('d')['year', 'month', 'day', 'FDE_Other02', 'FDE_B_System3',
       'FDE_C_System3', 'FDE_C_System4', 'FDE_B_System4', 'FDE_G_System3',
       'FDE_E_System4', 'FDE_A_System4', 'FDE_F_System4', 'FDE_G_System4',
       'FDE_F_System3', 'FDE_E_System1', 'FDE_A_System1', 'FDE_A_System3',
       'FDE_D_System3', 'FDE_Other10', 'FDE_Other11', 'FDE_Other12',
       'FDE_G_System2', 'FDE_Other03', 'FDE_Other00', 'FDE_Other04',
       'FDE_E_System3', 'FDE_Other01', 'FDE_G_System1', 'FDE_F_System1',
       'FDE_Other09', 'FDE_Other05', 'FDE_D_System4', 'FDE_H_System4',
       'FDE_H_System1', 'FDE_C_System2', 'FDE_B_System2', 'FDE_D_System1',
       'FDE_D_System2', 'FDE_Other06', 'FDE_K_System1', 'FDE_K_System4',
       'FDE_K_System2', 'FDE_Q_System3', 'FDE_C_System1', 'FDE_A_System2',
       'FDE_E_System2', 'FDE_H_System3', 'FDE_M_System3', 'FDE_F_System2',
       'FDE_Other18', 'FDE_B_System1', 'FDE_Other17', 'FDE_J_System3',
       'FDE_Other16', 'FDE_Other20', 'FDE_Other21', 'FDE_I_System2',
       'FDE_I_System3', 'FDE_N_System2', 'FDE_H_System2', 'FDE_P_System3',
       'FDE_Other15', 'FDE_Other07', 'FDE_Other08', 'FDE_M_System1',
       'FDE_N_System1', 'FDE_Other13', 'FDE_I_System1', 'FDE_M_System4',
       'FDE_N_System4', 'FDE_M_System2', 'FDE_Other19', 'FDE_J_System4',
       'FDE_Other14', 'FDE_P_System4', 'FDE_O_System2', 'FDE_O_System3',
       'FDE_J_System2', 'FDE_L_System1', 'FDE_J_System1'].asfreq().fillna(0).astype(int).reset_index()

In [44]:
# Exibir o cabeçalho do dataframe

df_FDE_All.head()

,Aircraft,Date,year,month,day,FDE_Other02,FDE_B_System3,FDE_C_System3,FDE_C_System4,FDE_B_System4,...,FDE_M_System2,FDE_Other19,FDE_J_System4,FDE_Other14,FDE_P_System4,FDE_O_System2,FDE_O_System3,FDE_J_System2,FDE_L_System1,FDE_J_System1
0,131,2006-04-29,2006,4,29,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,131,2006-04-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,131,2006-05-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,131,2006-05-02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,131,2006-05-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 1.2 Análise das Mensagens de Falha de um Aircraft Específico

In [45]:
# Definir uma variável para a seleção de um arcraft específico

var_aircraftSelected = 2766

In [46]:
# Criar um dataframe para o aircraft específico 

df_Failure_airSelec = df_FDE_All[df_FDE_All['Aircraft'] == var_aircraftSelected]

In [47]:
# Exibir o cabeçalho do dataframe

df_Failure_airSelec.head()

,Aircraft,Date,year,month,day,FDE_Other02,FDE_B_System3,FDE_C_System3,FDE_C_System4,FDE_B_System4,...,FDE_M_System2,FDE_Other19,FDE_J_System4,FDE_Other14,FDE_P_System4,FDE_O_System2,FDE_O_System3,FDE_J_System2,FDE_L_System1,FDE_J_System1
12377,2766,2006-04-19,2006,4,19,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12378,2766,2006-04-20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12379,2766,2006-04-21,2006,4,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12380,2766,2006-04-22,2006,4,22,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12381,2766,2006-04-23,2006,4,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_Failure_airSelec.shape

(1064, 82)

### 1.2.1 Gráfico de Série Temporal das Fault Text do Aircraft específico

In [27]:
array_data = []

for aux in array_FDE_All:
    
    trace = go.Bar(x = df_Failure_airSelec['Date'],
                   y = df_Failure_airSelec[aux],
                   name = aux,
                   marker = {'color': generate_color().upper()}) 
    
    # Adicionar o trace no array_data
    array_data.append(trace)
    
    layout = go.Layout(title='Fault Text Graphic',
                       xaxis=dict(tickfont=dict(size=14, color='rgb(107, 107, 107)')),
                       yaxis=dict(title='Quantity', titlefont=dict(size=16, color='rgb(107, 107, 107)'),
                       tickfont=dict(size=14, color='rgb(107, 107, 107)')), 
                       legend=dict(x=-0.5, y=-1.0, bgcolor='rgba(255, 255, 255, 0)',
                       bordercolor='rgba(255, 255, 255, 0)'),
                       barmode='group',
                       bargap=0.15,
                       bargroupgap=0.1)

    fig = dict(data=array_data, layout=layout) 

py.iplot(fig, filename='style-bar')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-499907cd38ae>", line 21, in <module>
    bargroupgap=0.1)
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\plotly\graph_objs\__init__.py", line 4948, in __init__
    self['geo'] = geo if geo is not None else _v
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\plotly\basedatatypes.py", line 3815, in __setitem__
    super(BaseLayoutHierarchyType, self).__setitem__(prop, value)
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\plotly\basedatatypes.py", line 2880, in __setitem__
    self._set_compound_prop(prop, value)
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\plotly\basedatatypes.py", line 3189, in _set_compound_prop
    val = validator.validate_coerce(val, skip_invalid=self._skip_invalid)
  File "C:\Users\Usuario\Anaconda3\lib\site-packages\_pl

KeyboardInterrupt: 

Imagem do gráfico interativo:
![title](plot_FDE.png)

# 2. Investigação Inicial da Estrutura e Conteúdo do Arquivo de Remoção

In [49]:
# Criar um dataframe para entrada de dados de remoção (arquivo do tipo .csv)

df_Removal = pd.read_csv('CBMpy_dataRemovalCode.csv')  

In [51]:
# Exibir do cabeçalho do dataframe

df_Removal.head()

,Aircraft,Component,System,Date,Reason,Time Hours,Time Cycles
0,1140,REM_Component_A,System1,2006-05-29,3,118123,15961
1,1140,REM_Component_A,System1,2006-05-29,3,118123,15961
2,1140,REM_Component_B,System1,2006-05-29,3,1092,139
3,1140,REM_Component_B,System3,2006-06-24,3,312,37
4,1140,REM_Component_B,System3,2006-07-10,3,118698,16028


In [52]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_Removal.shape

(1282, 7)

In [53]:
# Verificar o tipo de dado de cada coluna do dataframe

df_Removal.dtypes

Aircraft         int64
Component       object
System          object
Date            object
Reason           int64
Time Hours       int64
Time Cycles      int64
dtype: object

In [54]:
# Converter os dados da coluna 'Date' para o formato de data

df_Removal['Date'] =  pd.to_datetime(df_Removal['Date'], format='%Y/%m/%d')

In [55]:
# Verificar o tipo de dado de cada coluna do dataframe

df_Removal.dtypes

Aircraft                 int64
Component               object
System                  object
Date            datetime64[ns]
Reason                   int64
Time Hours               int64
Time Cycles              int64
dtype: object

In [56]:
# Determinar quantos 'Component' diferentes há no dataframe

len(df_Removal['Component'].unique())

17

In [57]:
# Verificar quais foram os 'Component' mais trocados 

df_Removal['Component'].value_counts()

REM_Component_B    268
REM_Component_A    210
REM_Component_D    177
REM_Component_F    113
REM_Component_J     87
REM_Component_G     86
REM_Component_H     77
REM_Component_I     64
REM_Component_N     45
REM_Component_E     38
REM_Component_L     37
REM_Component_K     25
REM_Component_O     22
REM_Component_M     19
REM_Component_C      6
REM_Component_P      5
REM_Component_Q      3
Name: Component, dtype: int64

In [58]:
# Verificar quais 'Aircraft' realizaram mais trocas de componentes  

df_Removal['Aircraft'].value_counts()

2640    99
2766    94
2361    92
2326    91
2567    86
1950    78
2982    74
1399    62
2838    60
2436    59
131     55
1151    54
312     53
1990    50
1419    49
736     46
1710    41
2209    38
791     37
326     30
1140    25
165      9
Name: Aircraft, dtype: int64

$\color{blue}{\text{OBSERVAÇÃO:}}$ O Aircraft igual a 2766 é o segundo que apresenta a maior quantidade de remoções de peças. Além disso, é o primeiro em relação a quantidade de Fault Text.

## 2.1 Agrupamento do Conjunto de Dados de Removal para todos os Aircraft

In [59]:
# Exibir do cabeçalho do dataframe

df_Removal.head()

,Aircraft,Component,System,Date,Reason,Time Hours,Time Cycles
0,1140,REM_Component_A,System1,2006-05-29,3,118123,15961
1,1140,REM_Component_A,System1,2006-05-29,3,118123,15961
2,1140,REM_Component_B,System1,2006-05-29,3,1092,139
3,1140,REM_Component_B,System3,2006-06-24,3,312,37
4,1140,REM_Component_B,System3,2006-07-10,3,118698,16028


In [60]:
# Identifica todas as "Fault Text" existentes de todos os Aircraft
array_Removal_All = np.array(df_Removal['Component'].unique())

# Exclui os itens NaN do array
array_Removal_All = array_Removal_All[~pd.isnull(array_Removal_All)]  

# Exibe o array
array_Removal_All

array(['REM_Component_A', 'REM_Component_B', 'REM_Component_C',
       'REM_Component_D', 'REM_Component_E', 'REM_Component_F',
       'REM_Component_G', 'REM_Component_H', 'REM_Component_I',
       'REM_Component_J', 'REM_Component_K', 'REM_Component_L',
       'REM_Component_M', 'REM_Component_N', 'REM_Component_O',
       'REM_Component_P', 'REM_Component_Q'], dtype=object)

In [61]:
# Realizar o merge do dataframe (agrupamento por data)

arrayY = []

df_Removal_All = pd.DataFrame(columns= ['Aircraft','year', 'month', 'day'])

for aux in array_Removal_All:
    
    # Cria um dataframe para um Fault Text corrente 
    dfMsg = pd.DataFrame(df_Removal[df_Removal['Component'] == aux])
    
    # Contar as Fault Text por dia para o aircraft selecionado
    arrayY = dfMsg.groupby([dfMsg['Aircraft'],
                            dfMsg['Date'].dt.year.rename('year'),
                            dfMsg['Date'].dt.month.rename('month'),
                            dfMsg['Date'].dt.day.rename('day')]).count()['Component']
    
    # Transformar os objetos do groupby para dataframe (depois possibilita fazer o merge).
    arrayY = arrayY.to_frame().reset_index()
    
    arrayY.columns = ['Aircraft', 'year', 'month', 'day', aux]
       
    # Utilizar o método "outer" (apropriado para acrescentar colunas e manter os índices compostos de dia-mes-ano). 
    df_Removal_All = pd.merge(df_Removal_All, arrayY, how='outer', on=['Aircraft', 'year','month','day'])

In [62]:
# Exibir o cabeçalho do dataframe

df_Removal_All.head()

,Aircraft,year,month,day,REM_Component_A,REM_Component_B,REM_Component_C,REM_Component_D,REM_Component_E,REM_Component_F,...,REM_Component_H,REM_Component_I,REM_Component_J,REM_Component_K,REM_Component_L,REM_Component_M,REM_Component_N,REM_Component_O,REM_Component_P,REM_Component_Q
0,131,2006,4,9,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,131,2007,9,15,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,131,2007,9,20,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,131,2007,10,6,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,131,2009,1,6,1.0,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Adicionar uma coluna date no dataframe (coo dados dos campos year, month e day) 

df_Removal_All['Date'] = pd.to_datetime(df_Removal_All.year*10000 + df_Removal_All.month*100 + df_Removal_All.day, format='%Y%m%d') 

In [64]:
# Ordena o dataframe por: year -> month -> day

df_Removal_All = df_Removal_All.sort_values(['year', 'month', 'day'])

In [65]:
# Substituir elementos NaN por zeros 

df_Removal_All = df_Removal_All.fillna(0) 

In [66]:
df_Removal_All.columns

Index(['Aircraft', 'year', 'month', 'day', 'REM_Component_A',
       'REM_Component_B', 'REM_Component_C', 'REM_Component_D',
       'REM_Component_E', 'REM_Component_F', 'REM_Component_G',
       'REM_Component_H', 'REM_Component_I', 'REM_Component_J',
       'REM_Component_K', 'REM_Component_L', 'REM_Component_M',
       'REM_Component_N', 'REM_Component_O', 'REM_Component_P',
       'REM_Component_Q', 'Date'],
      dtype='object')

In [67]:
# Preenche as datas faltantes 

df_Removal_All['Date'] = pd.to_datetime(df_Removal_All.Date)

df_Removal_All = df_Removal_All.set_index('Date')

df_Removal_All = df_Removal_All.groupby('Aircraft').resample('d')['year', 'month', 'day', 'REM_Component_A', 'REM_Component_B', 'REM_Component_C',
       'REM_Component_D', 'REM_Component_E', 'REM_Component_F',
       'REM_Component_G', 'REM_Component_H', 'REM_Component_I',
       'REM_Component_J', 'REM_Component_K', 'REM_Component_L',
       'REM_Component_M', 'REM_Component_N', 'REM_Component_O',
       'REM_Component_P', 'REM_Component_Q'].asfreq().fillna(0).astype(int).reset_index()

In [68]:
# Exibir o cabeçalho do dataframe

df_Removal_All.head()

,Aircraft,Date,year,month,day,REM_Component_A,REM_Component_B,REM_Component_C,REM_Component_D,REM_Component_E,...,REM_Component_H,REM_Component_I,REM_Component_J,REM_Component_K,REM_Component_L,REM_Component_M,REM_Component_N,REM_Component_O,REM_Component_P,REM_Component_Q
0,131,2006-03-19,2006,3,19,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,131,2006-03-20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,131,2006-03-21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,131,2006-03-22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,131,2006-03-23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 2.2 Análise das Remoções de um Aircraft Específico

In [69]:
# Definir uma variável para a seleção de um arcraft específico

#var_aircraftSelected = 2766

var_aircraftSelected

2766

In [70]:
# Criar um dataframe para o aircraft específico 

df_Removal_airSelec = df_Removal_All[df_Removal_All['Aircraft'] == var_aircraftSelected]

In [71]:
# Exibir o cabeçalho do dataframe

df_Removal_airSelec.head()

,Aircraft,Date,year,month,day,REM_Component_A,REM_Component_B,REM_Component_C,REM_Component_D,REM_Component_E,...,REM_Component_H,REM_Component_I,REM_Component_J,REM_Component_K,REM_Component_L,REM_Component_M,REM_Component_N,REM_Component_O,REM_Component_P,REM_Component_Q
19758,2766,2006-03-19,2006,3,19,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19759,2766,2006-03-20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19760,2766,2006-03-21,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19761,2766,2006-03-22,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19762,2766,2006-03-23,2006,3,23,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [72]:
# Verifica a dimensão do dataframe (qtd linhas, qtd colunas)

df_Removal_airSelec.shape

(1120, 22)

### 2.2.1 Gráfico de Série Temporal das Removal do Aircraft específico

In [73]:
array_data = []

for aux in array_Removal_All:
    
    trace = go.Bar(x = df_Removal_airSelec['Date'],
                   y = df_Removal_airSelec[aux],
                   name = aux,
                   marker = {'color': generate_color().upper()}) 
    
    # Adicionar o trace no array_data
    array_data.append(trace)
    
    layout = go.Layout(title='Removal Graphic',
                       xaxis=dict(tickfont=dict(size=14, color='rgb(107, 107, 107)')),
                       yaxis=dict(title='Quantity', titlefont=dict(size=16, color='rgb(107, 107, 107)'),
                       tickfont=dict(size=14, color='rgb(107, 107, 107)')), 
                       legend=dict(x=-0.5, y=-1.0, bgcolor='rgba(255, 255, 255, 0)',
                       bordercolor='rgba(255, 255, 255, 0)'),
                       barmode='group',
                       bargap=0.15,
                       bargroupgap=0.1)

    fig = dict(data=array_data, layout=layout) 

py.iplot(fig, filename='style-bar')

Imagem do gráfico interativo:
![title](plot_removal.png)

# 4. Detecção de falha 

Uma falha é caracterizada pela concentração frequente de FDEs.

In [74]:
# Exibir o cabeçalho do dataframe

df_FDE_All.head()

,Aircraft,Date,year,month,day,FDE_Other02,FDE_B_System3,FDE_C_System3,FDE_C_System4,FDE_B_System4,...,FDE_M_System2,FDE_Other19,FDE_J_System4,FDE_Other14,FDE_P_System4,FDE_O_System2,FDE_O_System3,FDE_J_System2,FDE_L_System1,FDE_J_System1
0,131,2006-04-29,2006,4,29,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,131,2006-04-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,131,2006-05-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,131,2006-05-02,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,131,2006-05-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# Verifica o array de todas as Fault Text

array_FDE_All

array(['FDE_Other02', 'FDE_B_System3', 'FDE_C_System3', 'FDE_C_System4',
       'FDE_B_System4', 'FDE_G_System3', 'FDE_E_System4', 'FDE_A_System4',
       'FDE_F_System4', 'FDE_G_System4', 'FDE_F_System3', 'FDE_E_System1',
       'FDE_A_System1', 'FDE_A_System3', 'FDE_D_System3', 'FDE_Other10',
       'FDE_Other11', 'FDE_Other12', 'FDE_G_System2', 'FDE_Other03',
       'FDE_Other00', 'FDE_Other04', 'FDE_E_System3', 'FDE_Other01',
       'FDE_G_System1', 'FDE_F_System1', 'FDE_Other09', 'FDE_Other05',
       'FDE_D_System4', 'FDE_H_System4', 'FDE_H_System1', 'FDE_C_System2',
       'FDE_B_System2', 'FDE_D_System1', 'FDE_D_System2', 'FDE_Other06',
       'FDE_K_System1', 'FDE_K_System4', 'FDE_K_System2', 'FDE_Q_System3',
       'FDE_C_System1', 'FDE_A_System2', 'FDE_E_System2', 'FDE_H_System3',
       'FDE_M_System3', 'FDE_F_System2', 'FDE_Other18', 'FDE_B_System1',
       'FDE_Other17', 'FDE_J_System3', 'FDE_Other16', 'FDE_Other20',
       'FDE_Other21', 'FDE_I_System2', 'FDE_I_System3',

In [76]:
# Definir um array de todos os Aircraft

array_Aircraft = np.array(df_FDE_All['Aircraft'].unique())

array_Aircraft

array([ 131,  312,  326,  791, 1151, 1419, 1710, 1950, 1990, 2209, 2436,
       2640, 2766, 2838, 2982], dtype=int64)

In [77]:
# A função func_date_init retorna um array de datas iniciais de concentrações de FDE

def func_date_init(array_date):  
    
    array_date_init_concentrations = []
    
    if(len(array_date) > 0):
    
        dates = sorted(array_date)
    
        concentrations = [[dates[0]]]                        # Inicializa com a primeira data

        for date in dates[1:]:                               # iterate through the rest of the dates
            last_date = concentrations[-1][-1]               # look at the last date we added
            if (date - last_date) <= pd.Timedelta(days=25):  # is it close enough to be in the same group?
                concentrations[-1].append(date)              # if so, then put it in the same group
            else:                                            # otherwise,
                concentrations.append([date])                # make a new group with it at the head
            
            
        for i in range(0, len(concentrations)):
            if(len(concentrations[i]) > 3):
                
                array_date_init_concentrations.append(concentrations[i][0])
    
    return(array_date_init_concentrations) 

In [78]:
# A func_date_end retorna um array de datas finais de concentrações de FDE

def func_date_end(array_date):   
    
    array_date_end_concentrations = []
    
    if(len(array_date) > 0):
    
        dates = sorted(array_date)
    
        concentrations = [[dates[0]]]                        # Inicializa com a primeira data

        for date in dates[1:]:                               # Iterar pelas datas restantes
            last_date = concentrations[-1][-1]               # Ver a última data adicionada
            if (date - last_date) <= pd.Timedelta(days=25):  # Está perto o suficiente para estar no mesmo grupo?
                concentrations[-1].append(date)              # Se sim, adicionar no mesmo grupo
            else:                                            # Caso contrário,
                concentrations.append([date])                # Cria um novo grupo na cabeça
            
            
        for i in range(0, len(concentrations)):

            if(len(concentrations[i]) > 3):
                tam = len(concentrations[i])
                array_date_end_concentrations.append(concentrations[i][tam-1])
    
    return(array_date_end_concentrations) 

In [80]:
# Criar/Inicializar os dicionários 
d  = {'Aircraft': [], 'FDE': [], 'Date Init': []}
d2 = {'Aircraft': [], 'FDE': [], 'Date End': []}
d3 = {'Aircraft': [], 'FDE': [], 'Date Init': [], 'Date End': []}

# Criar os dataframes
dfCorrente = pd.DataFrame(d)
dfCorrente_final = pd.DataFrame(d2)
dfMerge = pd.DataFrame(d3)
df_FDE_Resultado = pd.DataFrame(d3)

# Percorrer o array de Aircraft
for aircraft in array_Aircraft: 
    
    # Percorrer o array de FDE
    for fde in array_FDE_All:
        
        #zerar o array para a próxima iteração
        array_date = [] 
        
        # Percorrer as linhas da coluna FDE específica 
        for i in range(0, len(df_FDE_All[fde])):             
            
            if((df_FDE_All[fde].iloc[i] > 0) and (df_FDE_All["Aircraft"].iloc[i] == aircraft)):
                
                # Adiciona a data que a FDE apareceu
                array_date.append(df_FDE_All['Date'].iloc[i])
                
        #print(array_date) 
        
        # Chama a função de detecção de data de início de concentração de FDE   
        array_func = func_date_init(array_date)        
        
        # Chama a função de detecção de data final de concentração de FDE
        array_func_final = func_date_end(array_date)
        
        # O dfCorrente vai receber as colunas: Aircraft, FDE e as datas iniciais
        for j in range(0, len(array_func)):        
            dfCorrente.loc[j] = [aircraft, fde, array_func[j]]     
        
        
        # O dfCorrente_final vai receber as colunas: Aircraft, FDE e as datas finais 
        for k in range(0, len(array_func)):  
            dfCorrente_final.loc[k] = [aircraft, fde, array_func_final[k]]
        
        # Adicionar uma coluna auxiliar para possibilitar fazer a operação MERGE de forma correta
        dfCorrente['aux'] = dfCorrente.groupby(['Aircraft', 'FDE']).cumcount()
        dfCorrente_final['aux'] = dfCorrente_final.groupby(['Aircraft', 'FDE']).cumcount()
        
        # Realizar o MERGE entre os dois dataframes e excluir a coluna auxiliar
        dfMerge = pd.merge(dfCorrente,dfCorrente_final, on=['Aircraft', 'FDE', 'aux'],how='outer').drop('aux', axis=1)
   

        # Verificar se o dfCorrente não está vazio
        if((len(dfCorrente) > 0) and (len(dfCorrente)) == (len(dfCorrente_final))):
            
            df_FDE_Resultado = pd.concat([df_FDE_Resultado, dfMerge])
        
        # Zerar os arrays para a próxima iteração
        array_date = []
        array_func = []
        array_func_final = []
        
        # Zerar os dataframes para a próxima iteração
        d  = {'Aircraft': [], 'FDE': [], 'Date Init': []}
        d2 = {'Aircraft': [], 'FDE': [], 'Date End': []}
        d3 = {'Aircraft': [], 'FDE': [], 'Date Init': [], 'Date End': []}
        
        dfCorrente = pd.DataFrame(d)
        dfCorrente_final = pd.DataFrame(d2)        
        dfMerge = pd.DataFrame(d3)

In [81]:
# Converter a coluna 'Tail #' para int

df_FDE_Resultado['Aircraft'] = df_FDE_Resultado['Aircraft'].astype(int)

In [82]:
df_FDE_Resultado.head()

,Aircraft,FDE,Date Init,Date End
0,131,FDE_Other02,2006-08-15,2006-09-07
1,131,FDE_Other02,2008-10-09,2008-10-14
0,131,FDE_C_System4,2007-05-05,2007-05-21
0,131,FDE_B_System4,2007-05-05,2007-05-21
0,131,FDE_G_System2,2009-02-27,2009-03-03
